In [52]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [101]:
#Get All Years into Single DataFrame
import glob
import os
path = os.getcwd() + '/AllYears/'
allFiles = glob.glob(path + "/*.csv")

In [136]:
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    
    #some years have tabs at end of column names
    df.columns = [i.strip('\n') for i in df.columns]
    
    
    #Make sure SALE DATE column is a Date Type
    df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])
    
    #create year column
    df['YEAR'] = df['SALE DATE'].apply(lambda r: r.year)
    list_.append(df)
rawdf = pd.concat(list_,)

In [137]:
rawdf.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR
0,0,2,BATHGATE,01 ONE FAMILY HOMES,1,3030,70,,A1,4445 PARK AVENUE,,10457,1,0,1,1694,1497,1899,1,A1,0,2010-01-15,2010
1,1,2,BATHGATE,01 ONE FAMILY HOMES,1,3035,2,,S1,441 EAST 178 STREET,,10457,1,1,2,1287,2378,1899,1,S1,116000,2010-07-01,2010
2,2,2,BATHGATE,01 ONE FAMILY HOMES,1,3037,42,,A1,4428 PARK AVENUE,,10457,1,0,1,3525,1340,1899,1,A1,287000,2010-12-10,2010
3,3,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,64,,A1,467 EAST 185 STREET,,10458,1,0,1,1667,1296,1910,1,A1,150000,2010-04-20,2010
4,4,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,,A1,2085 BATHGATE AVENUE,,10457,1,0,1,2060,1629,1899,1,A1,145915,2010-07-27,2010


In [251]:
#cleanup DataFrame
#columns to keep
keep =(
[u'BOROUGH', u'NEIGHBORHOOD', u'BUILDING CLASS CATEGORY',
 u'TAX CLASS AT PRESENT', u'BLOCK', u'LOT',
       u'BUILDING CLASS AT PRESENT', u'ADDRESS',
       u'ZIP CODE', u'RESIDENTIAL UNITS', u'COMMERCIAL UNITS', u'TOTAL UNITS',
       u'LAND SQUARE FEET', u'GROSS SQUARE FEET', u'YEAR BUILT',
       u'TAX CLASS AT TIME OF SALE', u'BUILDING CLASS AT TIME OF SALE',
       u'SALE PRICE', u'SALE DATE', 'YEAR'])

df = rawdf[keep].reset_index(drop=True)

df = df[df['SALE PRICE'] > 1000]

In [236]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR
1,2,BATHGATE,01 ONE FAMILY HOMES,1,3035,2,S1,441 EAST 178 STREET,10457,1,1,2,1287,2378,1899,1,S1,116000,2010-07-01,2010
2,2,BATHGATE,01 ONE FAMILY HOMES,1,3037,42,A1,4428 PARK AVENUE,10457,1,0,1,3525,1340,1899,1,A1,287000,2010-12-10,2010
3,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,64,A1,467 EAST 185 STREET,10458,1,0,1,1667,1296,1910,1,A1,150000,2010-04-20,2010
4,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,A1,2085 BATHGATE AVENUE,10457,1,0,1,2060,1629,1899,1,A1,145915,2010-07-27,2010
5,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,42,A1,2069 BATHGATE AVENUE,10457,1,0,1,1964,1424,1899,1,A1,72790,2010-06-03,2010


In [151]:
df.count()

BOROUGH                           267379
NEIGHBORHOOD                      267379
BUILDING CLASS CATEGORY           267379
TAX CLASS AT PRESENT              267379
BLOCK                             267379
LOT                               267379
BUILDING CLASS AT PRESENT         267379
ADDRESS                           267379
ZIP CODE                          267379
RESIDENTIAL UNITS                 267379
COMMERCIAL UNITS                  267379
TOTAL UNITS                       267379
LAND SQUARE FEET                  267379
GROSS SQUARE FEET                 267379
YEAR BUILT                        267379
TAX CLASS AT TIME OF SALE         267379
BUILDING CLASS AT TIME OF SALE    267379
SALE PRICE                        267379
SALE DATE                         267379
YEAR                              267379
dtype: int64

In [257]:
#Only keep Zip Codes with data from all 4 Years
Num_Years = 5
EveryYear = (df.groupby('ZIP CODE')['YEAR'].nunique() == Num_Years)
zips = EveryYear[EveryYear == True].index

#Only keep Zip Codes with over 500 sales in the last 4 years
threshold = 500
Enough_Sales = (df.groupby('ZIP CODE').count() > threshold)
zips2 = groups[groups['BLOCK'] == True].index
print len(zips), len(zips2)

179 154


In [264]:
#Find intersection
keep_zips = np.intersect1d(zips,zips2)

In [265]:
df[df['ZIP CODE'].isin(keep_zips)]

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR
1,2,BATHGATE,01 ONE FAMILY HOMES,1,3035,2,S1,441 EAST 178 STREET,10457,1,1,2,1287,2378,1899,1,S1,116000,2010-07-01,2010
2,2,BATHGATE,01 ONE FAMILY HOMES,1,3037,42,A1,4428 PARK AVENUE,10457,1,0,1,3525,1340,1899,1,A1,287000,2010-12-10,2010
3,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,64,A1,467 EAST 185 STREET,10458,1,0,1,1667,1296,1910,1,A1,150000,2010-04-20,2010
4,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,A1,2085 BATHGATE AVENUE,10457,1,0,1,2060,1629,1899,1,A1,145915,2010-07-27,2010
5,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,42,A1,2069 BATHGATE AVENUE,10457,1,0,1,1964,1424,1899,1,A1,72790,2010-06-03,2010
6,2,BATHGATE,01 ONE FAMILY HOMES,1,3048,18,A9,2184 BATHGATE AVENUE,10457,1,0,1,1768,1236,1930,1,A9,313300,2010-04-09,2010
7,2,BATHGATE,01 ONE FAMILY HOMES,1,3048,19,A5,2186 BATHGATE AVENUE,10457,1,0,1,1768,1188,1901,1,A5,299000,2010-06-30,2010
8,2,BATHGATE,01 ONE FAMILY HOMES,1,3048,19,A5,2186 BATHGATE AVENUE,10457,1,0,1,1768,1188,1901,1,A5,132000,2010-02-23,2010
9,2,BATHGATE,01 ONE FAMILY HOMES,1,3048,28,A1,540 EAST 182 STREET,10457,1,0,1,1209,1048,1901,1,A1,272231,2010-10-04,2010
13,2,BATHGATE,02 TWO FAMILY HOMES,1,3030,60,B3,4469 PARK AVENUE,10457,2,0,2,1542,1542,1899,1,B3,88218,2010-01-27,2010
